In [ ]:
import os
import sqlite3
import enum

import pandas as pd
import lib
import dotenv

In [ ]:
dotenv.load_dotenv()

In [ ]:
class StatId(enum.Enum):
    population: str = "0000010101"
    environment: str = "0000010102"
    economics: str = "0000010103"
    administration: str = "0000010104"
    education: str = "0000010105"
    labour: str = "0000010106"
    culture: str = "0000010107"
    housing: str = "0000010108"
    medical_care: str = "0000010109"
    social_security: str = "0000010110"
    household_finance: str = "0000010111"
    daily_routine: str = "0000010112"

In [ ]:
conn = sqlite3.connect("./data/population.db")
df = pd.read_sql("SELECT * FROM record;", conn)
df.head()

In [ ]:
stats_res = lib.fetch(
    endpoint="getStatsData",
    params={
        "appId": os.getenv("APP_ID"),
        "statsDataId": StatId.population.value,
    },
)
stats_data = lib.StatsData(stats_res)

for i in stats_data.get_class():
    print(i)